<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/main/HomeWork_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets transformers

In [ ]:
import pandas as pd
import numpy as np
import torch 
import re

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelForCausalLM

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 14/dataset.jsonl'

In [ ]:
with open(DATASET_PATH) as f: # open the dataset file
    df = pd.read_json(DATASET_PATH, lines=True).set_index('id')

In [ ]:
df

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...
...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\ny..."
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, ж..."
463646,2020-11-26 06:46:00+00:00,88.0,#всебожьяроса\nxxx: Судьба айтишников вообще н...


In [ ]:
def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [ ]:
df['text'] = df['text'].apply(clear_text)

In [ ]:
data = df.loc[:, 'text']

In [ ]:
data

id
1           ppdv, все юниксы очень дружелюбны.. они прос...
2           а ты не чувствуешь красоту мира?   честно го...
3           "мышка, почему у тебя такие большие глаза?" ...
4           "мальчики, вы что больные, бегать в палату к...
5           мы - как разработчики - живём с субейзом под...
                                ...                        
463644    xxx: угадайте не гугля, что такое жопиздан! yy...
463645    xxx: посетила шальная мысль заняться собой, жи...
463646    #всебожьяроса xxx: судьба айтишников вообще не...
463647    прибывшие на место правоохранители установили,...
463648    ххх: вот ребята из английской фирмы tvr (давны...
Name: text, Length: 81497, dtype: object

In [ ]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

build_text_files(data,'dataset.txt')

In [ ]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
data_path = 'dataset.txt'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def load_dataset(data_path, tokenizer):
  dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=data_path,
          block_size=256)

  data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
  return dataset, data_collator


dataset, data_collator = load_dataset(data_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21169
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 10586


Step,Training Loss
500,4.078500
1000,4.033300
1500,4.007000
2000,3.987500
2500,3.965700
3000,3.957500
3500,3.962100
4000,3.923500
4500,3.913400
5000,3.920100


Saving model checkpoint to ./gpt2-chief/checkpoint-500
Configuration saved in ./gpt2-chief/checkpoint-500/config.json
Model weights saved in ./gpt2-chief/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-1000
Configuration saved in ./gpt2-chief/checkpoint-1000/config.json
Model weights saved in ./gpt2-chief/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-1500
Configuration saved in ./gpt2-chief/checkpoint-1500/config.json
Model weights saved in ./gpt2-chief/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-2000
Configuration saved in ./gpt2-chief/checkpoint-2000/config.json
Model weights saved in ./gpt2-chief/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-2500
Configuration saved in ./gpt2-chief/checkpoint-2500/config.json
Model weights saved in ./gpt2-chief/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-30

TrainOutput(global_step=10586, training_loss=3.829743656840956, metrics={'train_runtime': 3539.4608, 'train_samples_per_second': 11.962, 'train_steps_per_second': 2.991, 'total_flos': 5531291025408000.0, 'train_loss': 3.829743656840956, 'epoch': 2.0})

In [ ]:
def generate_text(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        do_sample=False,
        max_length=size+100, 
        repetition_penalty=5., 
        temperature=2,
        num_beams=10,
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result

In [ ]:
print(generate_text("xxx: программисты"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xxx: программисты, блин.  xxxx: а у нас в офисе на стене висит табличка с надписью "не курить" yyy: и что это значит? xxxx: я не курю =)  ххх: привет! как дела? ууу: да все нормально... ххх: давай познакомимся? ууу: о_о ххх: ты меня любишь? ууу: ммммм....у тебя есть девушка? ууу: нет


In [ ]:
print(generate_text("xxx: вчера пошел погулять"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xxx: вчера пошел погулять с собакой. xxxxx: а она на меня смотрит, как на идиота! yyyyyy: ну и что? xxxxx: я ей говорю "я не идиот"  ххх: у нас в школе была традиция - если кто-то из учеников писал контрольную по алгебре или геометрии, то это означало, что он был готов к тому, что его постигнет та же судьба, что и всех остальных ((( 


In [ ]:
print(generate_text("xxx: друг работает админом"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xxx: друг работает админом в техподдержке одного провайдера. xxxxx: он говорит, что у него нет доступа к интернету из-за того, что ему не хватает места на жестком диске... yyyy: ну и? xxxx: а то!  ххх: когда я учился в универе, мы с девчонками играли в компьютерные игры типа "угадай загадку" или "сделай так, чтобы все было по-твоему".


In [ ]:
print(generate_text("xxx: я пришел домой а там"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xxx: я пришел домой а там на столе записка с надписью "привет, как дела?" xxxxx: что-то в этом роде yyyy: и? xxxx: это был мой первый рабочий день!  ххх: у меня есть один знакомый программист. он не очень любит когда его спрашивают о чем-то постороннем (в том числе об компьютерах), но при этом никогда не обижается по поводу того, что кто-то может сказать ему то же самое. поэтому


In [ ]:
print(generate_text("xxx: меня бросила жена"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xxx: меня бросила жена. xxxxx: я не знаю, что делать yyyyy: в смысле? xxxx: у нас с женой разные взгляды на жизнь  ххх: а как ты относишься к тому, что после того, как твоя девушка рассталась с тобой, тебе становится стыдно за то, что она до сих пор не рассталась с тобой? ууу: честно говоря, это было бы гораздо проще, если бы моя девушка рассталась с кем-то другим


В целом модель очень хорошо справляется с генерацией